# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../WeatherPy/cities_weather_data.csv")
weather_data

,City,Cloud Coverage,City Country,City DateTime,Humidity,City Latitude,City Longitude,Max Temperature,Wind Speed
0,Faanui,100,PF,1637639093,74,-16.4833,-151.7500,81.34,8.84
1,Tuktoyaktuk,40,CA,1637639093,77,69.4541,-133.0374,-5.80,10.36
2,Hilo,90,US,1637639094,96,19.7297,-155.0900,73.13,4.61
3,Saint-Joseph,17,RE,1637639094,67,-21.3667,55.6167,77.36,4.83
4,Wagar,18,SD,1637639095,75,16.1525,36.2032,71.24,10.36
...,...,...,...,...,...,...,...,...,...
558,Kaeo,95,NZ,1637639329,65,-35.1000,173.7833,68.45,4.43
559,Camaná,100,PE,1637639329,86,-16.6228,-72.7111,65.61,5.28
560,Devils Lake,1,US,1637639330,79,48.1128,-98.8651,29.95,7.00
561,Coruripe,13,BR,1637639330,80,-10.1256,-36.1756,76.75,8.95


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Use Lat and Lng as locations and Humidity as the weight
locations = weather_data[["City Latitude", "City Longitude"]]
weight = weather_data["Humidity"]

In [18]:
# Add heatmap layer to map
# Customize the size of the figure
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# fig=gmaps.figure()
fig = gmaps.figure(layout=figure_layout, zoom_level=2, center=(0,0))
heatmap = gmaps.heatmap_layer(locations, weights=weight, dissipating=False, max_intensity=100, point_radius=3)
fig.add_layer(heatmap)

# Set maximum intensity and point_radius settings
#heatmap_layer.max_intensity = max(weight)
#heatmap_layer.point_radius = 3

# Display heatmap
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
# Narrow down the cities to find ideal weather conditions:
# 1 A max temperature lower than 80 degrees but higher than 70
# 2 Wind speed less than 10 mph
# 3 Zero cloudiness

cities_df = weather_data.loc[(weather_data["Max Temperature"] >= 70) & 
                            (weather_data["Max Temperature"] <= 80) & 
                            (weather_data["Wind Speed"] <= 10) & 
                            (weather_data["Cloud Coverage"] == 0), :].reset_index(drop=True).dropna()
cities_df


,City,Cloud Coverage,City Country,City DateTime,Humidity,City Latitude,City Longitude,Max Temperature,Wind Speed
0,Kota,0,IN,1637639183,32,25.1833,75.8333,74.21,2.24
1,Pāli,0,IN,1637639194,20,25.7667,73.3333,75.92,2.51
2,Santa Maria,0,BR,1637639020,73,-29.6842,-53.8069,74.50,2.30
3,Rājāpur,0,IN,1637639236,24,25.3833,81.1500,73.08,5.12
4,Tessaoua,0,NE,1637639237,22,13.7574,7.9874,74.93,8.14
5,Bilma,0,NE,1637639239,24,18.6853,12.9164,71.15,8.90
6,Agadez,0,NE,1637639246,23,19.7500,10.2500,70.45,9.82
7,Manga,0,BF,1637639252,23,11.6636,-1.0731,74.03,6.31
8,Pokaran,0,IN,1637639257,10,26.9167,71.9167,76.46,4.50
9,Córdoba,0,AR,1637639324,49,-31.4135,-64.1811,74.25,6.91


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [20]:
# Store into variable named hotel_df
hotel_df = cities_df[["City", "City Country", "City Latitude", "City Longitude"]].copy()

# Add "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""
hotel_df

,City,City Country,City Latitude,City Longitude,Hotel Name
0,Kota,IN,25.1833,75.8333,
1,Pāli,IN,25.7667,73.3333,
2,Santa Maria,BR,-29.6842,-53.8069,
3,Rājāpur,IN,25.3833,81.1500,
4,Tessaoua,NE,13.7574,7.9874,
5,Bilma,NE,18.6853,12.9164,
6,Agadez,NE,19.7500,10.2500,
7,Manga,BF,11.6636,-1.0731,
8,Pokaran,IN,26.9167,71.9167,
9,Córdoba,AR,-31.4135,-64.1811,


In [21]:
# Find the closest hotels; define base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Set parameters to search for hotels with 5000 meters.
params = {"search" : "hotel",
          "type" : "lodging",
          "radius" : 5000,
          "key" : g_key
}

In [22]:
# Loop over each city in the dataframe and make an API request

for index, row in hotel_df.iterrows():
    # Get city name, lat, lng from df
    lat = row["City Latitude"]
    lng = row["City Longitude"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # Request the API and get hotel details for each city 
    response = requests.get(base_url, params).json()
    
    try:
        hotel_name = response['results'][0]['name']
        hotel_df.loc[index, "Hotel Name"] = hotel_name
        
    except IndexError:
        hotel_df.loc[index, "Hotel Name"] = 'No hotel found'


In [23]:
hotel_df

,City,City Country,City Latitude,City Longitude,Hotel Name
0,Kota,IN,25.1833,75.8333,Capital O 84094 Hotel Tanishq
1,Pāli,IN,25.7667,73.3333,HOTEL THE AMBA VILAS
2,Santa Maria,BR,-29.6842,-53.8069,Hotel Morotin
3,Rājāpur,IN,25.3833,81.1500,Ravi roy
4,Tessaoua,NE,13.7574,7.9874,Benin Dan Kaabou Heberge
5,Bilma,NE,18.6853,12.9164,Expeditions Ténére voyages
6,Agadez,NE,19.7500,10.2500,No hotel found
7,Manga,BF,11.6636,-1.0731,Auberge du 11 décembre de la cité CNSS à Manga
8,Pokaran,IN,26.9167,71.9167,Motel Pokaran - RTDC
9,Córdoba,AR,-31.4135,-64.1811,Hotel Sussex


In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>City Country</dt><dd>{City Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["City Latitude", "City Longitude"]]

In [25]:
# Add marker layer ontop of heat map

# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…